In [ ]:
!pip install -q transformers torch pandas numpy scikit-learn tqdm matplotlib huggingface_hub peft

import os, subprocess, time

def stage_to_local(src_dir: str, dst_dir: str, marker_name=".staged_ok"):
    os.makedirs(dst_dir, exist_ok=True)
    marker = os.path.join(dst_dir, marker_name)

    # 이미 스테이징 완료면 스킵
    if os.path.exists(marker):
        print(f"[stage] already staged: {dst_dir}")
        return

    print(f"[stage] rsync from\n  {src_dir}\n  -> {dst_dir}")
    t0 = time.time()

    # rsync는 증분 복사라서 중간에 끊겨도 다시 실행하면 이어서 빨리 맞춰짐
    cmd = f"rsync -a --info=progress2 '{src_dir.rstrip('/')}/' '{dst_dir.rstrip('/')}/'"
    subprocess.run(["bash", "-lc", cmd], check=True)

    # 마커 생성
    with open(marker, "w") as f:
        f.write("ok\n")

    print(f"[stage] done in {time.time()-t0:.1f}s")

# ====== 0) 구글드라이브 마운트 & code 경로 설정 ======
import os, sys

# Colab이 아니면 drive mount가 실패할 수 있음(로컬/엘리스/기타 환경 호환)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('[warn] drive mount skipped:', e)

# 프로젝트 루트(Drive 기준). 필요하면 여기만 바꾸면 됨.
DRIVE_ROOT = '/content/drive/MyDrive/TRAITHON'
DRIVE_CODE = os.path.join(DRIVE_ROOT, 'code')
DRIVE_ADAPTER = os.path.join(DRIVE_ROOT, 'models/adapter')

# /content 로컬 스테이징(빠름)
LOCAL_CODE       = '/content/code'
LOCAL_BASE_MODEL = '/content/A.X-4.0-Light'
LOCAL_ADAPTER    = '/content/ax4_adapter_final'

# (선택) Drive에 있는 모듈/어댑터를 /content로 스테이징 (rsync 기반, 증분 복사)
if os.path.exists(DRIVE_CODE):
    try:
        stage_to_local(DRIVE_CODE, LOCAL_CODE)
    except Exception as e:
        print('[warn] code staging failed:', e)

if os.path.exists(DRIVE_ADAPTER):
    try:
        stage_to_local(DRIVE_ADAPTER, LOCAL_ADAPTER)
    except Exception as e:
        print('[warn] adapter staging failed:', e)

# 실제로 import에 쓸 CODE_DIR 결정 (우선순위: /content/code -> Drive/code -> ./code)
if os.path.exists(LOCAL_CODE):
    CODE_DIR = LOCAL_CODE
elif os.path.exists(DRIVE_CODE):
    CODE_DIR = DRIVE_CODE
else:
    CODE_DIR = './code'

if CODE_DIR not in sys.path:
    sys.path.insert(0, CODE_DIR)

print('CODE_DIR =', CODE_DIR)
print('LOCAL_ADAPTER =', LOCAL_ADAPTER)

# 데이터셋 경로 추가 설정
DRIVE_DATA = os.path.join(DRIVE_ROOT, 'data/golden_set_v1_TL')
LOCAL_DATA = '/content/data'

if os.path.exists(DRIVE_DATA):
    try:
        # 데이터셋 5,000개를 /content/data로 복사 (이게 가장 중요!)
        stage_to_local(DRIVE_DATA, LOCAL_DATA)
    except Exception as e:
        print('[warn] data staging failed:', e)

# 이후 6번째 셀에서 DATA_PATH를 LOCAL_DATA로 바꿔서 사용하세요.
DATA_PATH = LOCAL_DATA

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[stage] already staged: /content/code
[stage] already staged: /content/ax4_adapter_final
CODE_DIR = /content/code
LOCAL_ADAPTER = /content/ax4_adapter_final
[stage] already staged: /content/data


In [ ]:
from hf_ax4_downloader import download_ax4_light_to_content

model_dir = download_ax4_light_to_content("/content/A.X-4.0-Light")
print("MODEL DIR =", model_dir)

[hf_dl] already ready: /content/A.X-4.0-Light
MODEL DIR = /content/A.X-4.0-Light


In [ ]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# 앞선 셀에서 설정된 경로 변수 사용
# LOCAL_BASE_MODEL, LOCAL_ADAPTER, CODE_DIR

# 1. 모듈 임포트 (전처리 모듈 포함 필수)
try:
    import ax4_clickbait_scorer as sc
    import clickbait_preprocess as cp  # ★ 핵심: 동료 코드와 동일한 전처리기
    print(f"✅ 모듈 로드 성공: {sc.__file__}, {cp.__file__}")
except ImportError as e:
    print(f"🚨 임포트 에러: {e}")
    print(f"현재 sys.path: {sys.path}")
    sys.exit()

# 2. 모델 초기화 (메모리 정리 후 어댑터 장착)
sc._MODEL = None
sc._TOKENIZER = None
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"모델 로딩 중...\n Base: {LOCAL_BASE_MODEL}\n Adapter: {LOCAL_ADAPTER}")

# 어댑터 경로 유효성 재확인
if not os.path.exists(LOCAL_ADAPTER) or not os.listdir(LOCAL_ADAPTER):
    print("⚠️ 경고: 로컬 어댑터 폴더가 비어있거나 없습니다. 확인 필요!")
else:
    # init_model 호출
    sc.init_model(model_path=LOCAL_BASE_MODEL, adapter_path=LOCAL_ADAPTER)
    print("🎉 모델 로드 및 어댑터 장착 완료.")

    if hasattr(sc._MODEL, "merge_and_unload"):
        print("🚀 어댑터 머지 시작 (Inference 속도 최적화)...")
        sc._MODEL = sc._MODEL.merge_and_unload()
        print("🎉 머지 완료. 이제 단일 모델로 동작합니다.")
    else:
        print("⚠️ 머지할 어댑터가 없거나 이미 머지된 상태입니다.")

✅ 모듈 로드 성공: /content/code/ax4_clickbait_scorer.py, /content/code/clickbait_preprocess.py
모델 로딩 중...
 Base: /content/A.X-4.0-Light
 Adapter: /content/ax4_adapter_final
[ax4_clickbait_scorer] Loading tokenizer from: /content/ax4_adapter_final
[ax4_clickbait_scorer] Loading base model from: /content/A.X-4.0-Light
[ax4_clickbait_scorer] device=cuda, dtype=torch.bfloat16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[ax4_clickbait_scorer] Attaching adapter (NO MERGE): /content/ax4_adapter_final
[ax4_clickbait_scorer] Model loaded.
[ax4_clickbait_scorer] Adapter active: /content/ax4_adapter_final
[ax4_clickbait_scorer] run_meta loaded: ans0=0(id=56), ans1=1(id=57)
🎉 모델 로드 및 어댑터 장착 완료.
🚀 어댑터 머지 시작 (Inference 속도 최적화)...
🎉 머지 완료. 이제 단일 모델로 동작합니다.


In [1]:
import random
import unicodedata

class ObfuscationAttacker:
    def __init__(self):
        self.f1_chars = ['\u200B', '\u200C', '\u200D', '\u200E', '\u200F', '\ufeff']
        self.f2_map = {
            ' ': ['\u2000', '\u2001', '\u1680', '\u202f', '\u205f', '\u3000'],
            '.': ['．', '․', '…'], ',': ['，', '､'], '!': ['！', '﹗']
        }
        self.f3_map = {'a': 'а', 'e': 'е', 'o': 'о', 'p': 'р', 'c': 'с', 'y': 'у', 'd': 'ԁ', 'i': 'і'}
        self.f4_marks = ['\u0300', '\u0301', '\u0308', '\u0327', '\u0332', '\u0311']

    def apply_attack(self, text, family, level='L2'):
        prob = {'L1': 0.001, 'L2': 0.003, 'L3': 0.01}[level]
        text = unicodedata.normalize('NFKC', text)
        chars = list(text)
        new_chars = []

        if family == 'F1':
            for c in chars:
                new_chars.append(c)
                if random.random() < prob: new_chars.append(random.choice(self.f1_chars))
        elif family == 'F2':
            for c in chars:
                if c in self.f2_map and random.random() < prob * 10:
                    new_chars.append(random.choice(self.f2_map[c]))
                else: new_chars.append(c)
        elif family == 'F3':
            for c in chars:
                c_low = c.lower()
                if c_low in self.f3_map and random.random() < prob * 5:
                    sub = self.f3_map[c_low]
                    new_chars.append(sub.upper() if c.isupper() else sub)
                else: new_chars.append(c)
        elif family == 'F4':
            for c in chars:
                new_chars.append(c)
                if not c.isspace() and random.random() < prob:
                    new_chars.append(random.choice(self.f4_marks))
        return "".join(new_chars)

attacker = ObfuscationAttacker()

In [2]:
import math
import unicodedata

def get_mixed_script_ratio(text):
    """한글, 라틴, 숫자, 문장부호 이외의 문자가 포함된 비율을 계산"""
    if not text: return 0.0
    total_len = len(text)
    extra_chars = 0
    for char in text:
        cat = unicodedata.category(char)
        # 공백(Zs), 문장부호(P*) 제외
        if cat.startswith('Z') or cat.startswith('P'):
            continue
        name = unicodedata.name(char, "")
        # 한글, 라틴(영어), 숫자 이외의 스크립트가 포함되었는지 확인
        if not any(x in name for x in ["HANGUL", "LATIN", "DIGIT"]):
            extra_chars += 1
    return extra_chars / total_len

# def calculate_ppl(text):
#     if not text.strip(): return 0.0
#     encodings = sc._TOKENIZER(text, return_tensors="pt").to(sc._DEVICE)
#     with torch.no_grad():
#         outputs = sc._MODEL(encodings.input_ids, labels=encodings.input_ids)
#         loss = outputs.loss.item()
#     return math.exp(loss) if loss < 20 else 1e6

# def get_adv_metrics(text):
#     item = {'newsTitle': '', 'newsContent': text, 'newsCategory': 'ETC'}
#     score_res = sc.score_article(item)

#     num_chars = len(text)
#     token_ids = sc._TOKENIZER.encode(text)
#     num_tokens = len(token_ids)

#     tpc = num_tokens / num_chars if num_chars > 0 else 0
#     ppl = calculate_ppl(text)
#     ms_ratio = get_mixed_script_ratio(text)

#     return {
#         'score': score_res['p_clickbait'],
#         'ppl': ppl,
#         'tpc': tpc,
#         'tokens': num_tokens,
#         'ms_ratio': ms_ratio
#     }

# 2. 배치 PPL 계산 함수 (핵심 수정 사항)
def calculate_ppl_batch(texts, batch_size=8):
    """문장 리스트를 받아 개별 PPL을 한꺼번에 반환"""
    if not texts: return []

    all_ppls = []
    sc._MODEL.eval()

    # 토크나이저 패딩 설정 (배치 처리를 위해 필수)
    if sc._TOKENIZER.pad_token is None:
        sc._TOKENIZER.pad_token = sc._TOKENIZER.eos_token

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        # padding=True로 설정하여 배치의 문장 길이를 맞춤
        encodings = sc._TOKENIZER(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(sc._DEVICE)

        with torch.no_grad():
            outputs = sc._MODEL(encodings.input_ids, labels=encodings.input_ids)
            logits = outputs.logits
            labels = encodings.input_ids

            # 개별 문장별 Loss를 구하기 위한 연산 (Reduction='none')
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
            # [Batch, Seq_len] 형태의 개별 토큰별 Loss 획득
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss = loss.view(shift_labels.size())

            # 패딩 토큰은 계산에서 제외(Masking)
            mask = (shift_labels != sc._TOKENIZER.pad_token_id)
            sentence_loss = (loss * mask).sum(1) / mask.sum(1)

            for l in sentence_loss:
                all_ppls.append(math.exp(l.item()))

    return all_ppls

def get_adv_metrics_batch(texts):
    """문장 리스트를 받아 여러 지표를 배치로 반환"""
    # 1. 낚시성 점수 (기존 함수 활용 - 내부에서 반복문 돌 수 있음)
    scores = [sc.score_article({'newsTitle': '', 'newsContent': t, 'newsCategory': 'ETC'})['p_clickbait'] for t in texts]

    # 2. 배치 PPL 계산 (가장 무거운 작업)
    ppls = calculate_ppl_batch(texts)

    # 3. 기타 지표 계산
    results = []
    for i, text in enumerate(texts):
        num_chars = len(text)
        token_ids = sc._TOKENIZER.encode(text)
        num_tokens = len(token_ids)
        tpc = num_tokens / num_chars if num_chars > 0 else 0
        ms_ratio = get_mixed_script_ratio(text)

        results.append({
            'score': scores[i],
            'ppl': ppls[i],
            'tpc': tpc,
            'tokens': num_tokens,
            'ms_ratio': ms_ratio
        })
    return results

def perform_cleanup(text, family):
    if family == 'F1':
        return "".join([c for c in text if unicodedata.category(c) != 'Cf'])
    elif family == 'F4':
        return "".join([c for c in text if unicodedata.category(c) not in ['Mn', 'Mc', 'Me']])
    return text

In [3]:
import json
from tqdm.auto import tqdm

# 로컬 스테이징을 했다면 LOCAL_DATA, 아니면 DRIVE_DATA 사용
DATA_PATH = LOCAL_DATA

json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')]
# json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')][:5]
obf_results = []

print(f"🚀 총 {len(json_files)}개 파일 배치 실험 시작 (Batch Size: 1 File's All Variants)")

for i, filename in enumerate(tqdm(json_files)):
    file_path = os.path.join(DATA_PATH, filename)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)

        # 1. 텍스트 변종들 생성 (한꺼번에 배치 처리하기 위함)
        processed = cp.preprocess_article(raw_data)
        category = processed.get('newsCategory', 'ETC')
        orig_text = (processed['newsTitle'] + " " + processed['newsContent']).strip()

        # Axis B용 RAW 데이터
        src_info = raw_data.get("sourceDataInfo", {})
        raw_text = (src_info.get("newsTitle", "") + " " + src_info.get("newsContent", "")).strip()

        # 배치에 넣을 텍스트 리스트와 메타데이터 매핑용 리스트
        batch_texts = [orig_text]
        meta_info = [('ORIG', '-')]

        # F1~F4 공격 및 클린업 텍스트 생성
        attack_variants = {} # { (fam, lvl): adv_idx, (fam, lvl, 'clean'): clean_idx }

        for fam in ['F1', 'F2', 'F3', 'F4']:
            for lvl in ['L1', 'L3']:
                adv_text = attacker.apply_attack(orig_text, fam, lvl)
                batch_texts.append(adv_text)
                attack_variants[(fam, lvl)] = len(batch_texts) - 1
                meta_info.append((fam, lvl))

                # F1, F4인 경우 클린업 버전도 배치에 추가
                if fam in ['F1', 'F4']:
                    cleaned_text = perform_cleanup(adv_text, fam)
                    batch_texts.append(cleaned_text)
                    attack_variants[(fam, lvl, 'clean')] = len(batch_texts) - 1
                    meta_info.append((fam, lvl, 'CLEAN'))

        if raw_text:
            batch_texts.append(raw_text)
            meta_info.append(('F5_RAW', 'N/A'))

        # 2. ★ 배치 지표 한꺼번에 계산 (파일당 약 14~15개 문장)
        batch_metrics = get_adv_metrics_batch(batch_texts)

        # 3. 결과 정리 및 Gain 계산
        # 원본 지표
        orig_m = batch_metrics[0]
        obf_results.append({
            'id': filename, 'category': category, 'family': 'ORIG', 'level': '-',
            **orig_m, 'gain_ppl': 0, 'gain_tok': 0
        })

        # 공격군 지표
        for fam in ['F1', 'F2', 'F3', 'F4']:
            for lvl in ['L1', 'L3']:
                adv_m = batch_metrics[attack_variants[(fam, lvl)]]

                gain_ppl, gain_tok = 0, 0
                if fam in ['F1', 'F4']:
                    clean_m = batch_metrics[attack_variants[(fam, lvl, 'clean')]]
                    gain_ppl = adv_m['ppl'] - clean_m['ppl']
                    gain_tok = adv_m['tokens'] - clean_m['tokens']

                obf_results.append({
                    'id': filename, 'category': category, 'family': fam, 'level': lvl,
                    **adv_m, 'gain_ppl': gain_ppl, 'gain_tok': gain_tok
                })

        # F5 RAW 지표
        if raw_text:
            raw_m = batch_metrics[-1]
            obf_results.append({
                'id': filename, 'category': category, 'family': 'F5_RAW', 'level': 'N/A',
                **raw_m, 'gain_ppl': 0, 'gain_tok': 0
            })

        # 500개마다 중간 저장
        if (i + 1) % 500 == 0:
            pd.DataFrame(obf_results).to_csv(f'obf_batch_progress_{i+1}.csv', index=False)

    except Exception as e:
        print(f"🚨 Error in {filename}: {e}")

# 최종 결과 저장
df_obf = pd.DataFrame(obf_results)
df_obf.to_csv('obf_experiment_5000_batch_final.csv', index=False)
print(f"✅ 실험 완료: {len(df_obf)}행 데이터 수집됨.")

NameError: name 'LOCAL_DATA' is not defined

In [ ]:
# 1. 섹션별 베이스라인(ORIG) 계산
df_orig_base = df_obf[df_obf['family'] == 'ORIG'].groupby('category').agg({
    'ppl': 'mean',
    'tpc': 'mean',
    'ms_ratio': 'mean'
}).rename(columns={'ppl': 'base_ppl', 'tpc': 'base_tpc', 'ms_ratio': 'base_ms'})

# 2. 베이스라인 병합
df_merged = df_obf.merge(df_orig_base, on='category')

# 3. 정규화 지표 계산 (Ratio & Delta)
df_merged['ppl_ratio'] = df_merged['ppl'] / df_merged['base_ppl']
df_merged['tpc_ratio'] = df_merged['tpc'] / df_merged['base_tpc']
df_merged['ms_delta'] = df_merged['ms_ratio'] - df_merged['base_ms']

# 4. 전체 요약 (Family/Level별)
summary = df_merged[df_merged['family'] != 'ORIG'].groupby(['family', 'level']).agg({
    'ppl_ratio': 'mean',
    'tpc_ratio': 'mean',
    'ms_delta': 'mean',
    'gain_tok': 'mean',  # 토큰 절감량(클린업 게인)
    'score': 'mean'
})

print("📊 [전체 공격 Family별 탐지 Signature 요약]")
print(summary)

# 5. 섹션별 취약점 분석 (슬라이스 정규화 검증 + 클린업 게인 추가)
section_summary = df_merged[df_merged['family'] != 'ORIG'].groupby(['category', 'family']).agg({
    'ppl_ratio': 'mean',
    'tpc_ratio': 'mean',
    'ms_delta': 'mean',
    'gain_tok': 'mean'
}).unstack()

print("\n📊 [섹션별 탐지 지표 종합 분석 (PPL/TPC Ratio, MS Delta, Cleanup Gain)]")
display(section_summary)

# 결과 저장
summary.to_csv('obf_final_summary.csv')
section_summary.to_csv('obf_section_analysis.csv')

📊 [전체 공격 Family별 탐지 Signature 요약]
              ppl_ratio  tpc_ratio  ms_delta  gain_tok     score
family level                                                    
F1     L1      1.053035   1.003201  0.000817    2.4242  0.459108
       L3      1.320950   1.040210  0.009635   24.3606  0.456973
F2     L1      1.105483   1.010927 -0.000163    0.0000  0.458287
       L3      1.445494   1.119748 -0.000163    0.0000  0.451300
F3     L1      1.006911   0.999192 -0.000128    0.0000  0.459343
       L3      1.018073   1.000208  0.000158    0.0000  0.459366
F4     L1      1.048218   1.001761  0.000610    1.6620  0.459161
       L3      1.359341   1.025571  0.007429   16.3272  0.458429
F5_RAW N/A     0.944764   1.001162  0.000016    0.0000  0.017027

📊 [섹션별 탐지 지표 종합 분석 (PPL/TPC Ratio, MS Delta, Cleanup Gain)]


ppl_ratio                                         tpc_ratio  \
family          F1        F2        F3        F4    F5_RAW        F1   
category                                                               
EC        1.191729  1.287116  1.014377  1.215292  0.948918  1.021811   
ET        1.183899  1.279218  1.011304  1.195422  0.930572  1.021267   
GB        1.198049  1.297748  1.017059  1.211785  0.938167  1.021780   
IS        1.183814  1.271522  1.018922  1.205660  0.939326  1.022346   
LC        1.160622  1.223673  1.006752  1.180606  0.972899  1.021369   
PO        1.190910  1.267225  1.009732  1.202710  0.958223  1.021638   
SO        1.190222  1.283795  1.009222  1.206166  0.934173  1.021649   

                                                  ms_delta            \
family          F2        F3        F4    F5_RAW        F1        F2   
category                                                               
EC        1.063135  1.000064  1.014255  1.001601  0.005082 -0.000264   
ET        1.065256  1.000029  1.013026  1.001405  0.005408 -0.000017   
GB        1.066079  0.999356  1.013278  1.001761  0.005366 -0.000110   
IS        1.062868  0.999905  1.013880  1.001158  0.005117 -0.000314   
LC        1.066200  0.999748  1.013305  1.000872  0.005223 -0.000119   
PO        1.069377  0.999170  1.013708  1.000882  0.005303 -0.000052   
SO        1.064664  0.999723  1.013942  1.000594  0.005131 -0.000223   

                                         gain_tok                              
family          F3        F4    F5_RAW         F1   F2   F3         F4 F5_RAW  
category                                                                       
EC       -0.000052  0.003980  0.000061  12.246228  0.0  0.0   8.403292    0.0  
ET        0.000237  0.004112 -0.000105  10.056856  0.0  0.0   6.525920    0.0  
GB        0.000076  0.004025  0.000090  12.448187  0.0  0.0   8.229275    0.0  
IS        0.000023  0.003910 -0.000041  14.390698  0.0  0.0   9.672868    0.0  
LC       -0.000004  0.003967 -0.000045  16.218284  0.0  0.0  10.626866    0.0  
PO        0.000010  0.004165  0.000063  16.617483  0.0  0.0  11.204196    0.0  
SO       -0.000109  0.003984  0.000035  12.491542  0.0  0.0   8.602488    0.0

In [ ]:
import pandas as pd
import numpy as np

def find_specialized_thresholds(df, target_fpr=3):
    """
    각 공격군(F1~F4)에 대해 개별적인 최적 임계값을 탐색.
    목표: 통합 오탐률(Total FPR)을 target_fpr 이하로 유지하며 개별 Recall 극대화.
    """
    print(f"🔍 [공격군별 최적화] 탐색 시작 (목표 통합 FPR < {target_fpr}%)\n")

    # 탐색 범위 설정
    g_steps = [5, 10, 15, 20]          # F1, F4용 Gain Threshold
    p_steps = [1.3, 1.4, 1.5, 2.0]     # F2용 PPL Ratio
    t_steps = [1.05, 1.1, 1.15, 1.2]   # F2용 TPC Ratio
    m_steps = [0.0001, 0.001, 0.005, 0.01] # F3용 MS Delta

    best_overall_recall = -1
    best_config = {}

    # 4중 루프를 통해 최적의 조합 탐색
    for g in g_steps:
        for p in p_steps:
            for t in t_steps:
                for m in m_steps:
                    # 1. 통합 오탐률(Total FPR) 계산
                    # 로직: (F1/F4 조건) OR (F2 조건) OR (F3 조건) 중 하나라도 걸리면 오탐
                    orig = df[df['family'] == 'ORIG']
                    is_fp = (
                        (orig['gain_tok'] >= g) |                      # F1, F4 오탐
                        ((orig['ppl_ratio'] >= p) & (orig['tpc_ratio'] >= t)) | # F2 오탐
                        (orig['ms_delta'] >= m)                        # F3 오탐
                    )
                    fpr = (is_fp.sum() / len(orig)) * 100 if len(orig) > 0 else 0

                    if fpr <= target_fpr:
                        # 2. 각 공격군별 개별 Recall 계산
                        recalls = {}
                        for f in ['F1', 'F2', 'F3', 'F4']:
                            sub = df[(df['family'] == f) & (df['level'] == 'L3')]
                            if len(sub) == 0: continue

                            if f == 'F1' or f == 'F4':
                                detected = sub[sub['gain_tok'] >= g]
                            elif f == 'F2':
                                detected = sub[(sub['ppl_ratio'] >= p) & (sub['tpc_ratio'] >= t)]
                            elif f == 'F3':
                                detected = sub[sub['ms_delta'] >= m]

                            recalls[f] = (len(detected) / len(sub)) * 100

                        avg_recall = np.mean(list(recalls.values()))

                        if avg_recall > best_overall_recall:
                            best_overall_recall = avg_recall
                            best_config = {
                                'params': {'g': g, 'p': p, 't': t, 'm': m},
                                'fpr': fpr,
                                'recalls': recalls,
                                'avg_recall': avg_recall
                            }

    # 최종 결과 출력
    if best_config:
        p = best_config['params']
        print(f"✨ [최적 탐지 시나리오 발견]")
        print(f"1. F1 & F4 전용 (Gain): {p['g']}개 이상 제거 시 탐지")
        print(f"2. F2 전용 (Context): PPL {p['p']}배 AND TPC {p['t']}배 동시 충족 시 탐지")
        print(f"3. F3 전용 (Script): MS Delta {p['m']} 이상 변동 시 탐지")
        print("-" * 50)
        print(f"📊 통합 오탐률(FPR): {best_config['fpr']:.2f}%")
        print(f"📈 평균 검출률(L3 Recall): {best_config['avg_recall']:.2f}%")
        for f, r in best_config['recalls'].items():
            print(f"   - {f} 탐지 성공률: {r:.2f}%")
    else:
        print("❌ 목표 FPR 내에서 적절한 임계값을 찾지 못했습니다. 범위를 조정하세요.")

# 최적화 실행
find_specialized_thresholds(df_merged)

🔍 [공격군별 최적화] 탐색 시작 (목표 통합 FPR < 3%)

✨ [최적 탐지 시나리오 발견]
1. F1 & F4 전용 (Gain): 5개 이상 제거 시 탐지
2. F2 전용 (Context): PPL 1.3배 AND TPC 1.1배 동시 충족 시 탐지
3. F3 전용 (Script): MS Delta 0.01 이상 변동 시 탐지
--------------------------------------------------
📊 통합 오탐률(FPR): 2.70%
📈 평균 검출률(L3 Recall): 55.65%
   - F1 탐지 성공률: 98.16%
   - F2 탐지 성공률: 28.64%
   - F3 탐지 성공률: 2.38%
   - F4 탐지 성공률: 93.40%
